# Vision - Walk datasets

## Python imports

In [24]:
import csv
import cv2
import os.path
import pandas as pd
import re
import VideoCapture

## Parameters

In [25]:
DEVICE = 'Xiaomi'

DPARAM = {
    'Pixel' : {
        'video': '../dados_indoor_location/Pixel/Vision/1228187371660504_teste_final.mp4',
        'walk_times': '../dados_indoor_location/walks/Walk_Pixel.tsv'
    },
    'Xiaomi': {
        'video': '../dados_indoor_location/Xiaomi/Vision/97082214722162_teste_final.3gp',
        'walk_times': '../dados_indoor_location/walks/Walk_Xiaomi.tsv'        
    }
}
TIME_GRANULARITY = 1_000_000_000
MERGE_ATRIO_LABELS = False

## Read walk data

In [26]:
time_re = re.compile('^(\d+):(\d+)$')
times = []
with open(DPARAM[DEVICE]['walk_times'], newline='') as csv_input:
    data = csv.reader(csv_input, delimiter='\t')
    for row in data:
      (begin,end,label) = row[1:4]
      # print(begin,end,label)
      m = time_re.match(begin)
      begin = int(m.group(1)) * 60 + int(m.group(2)) 
      m = time_re.match(end)
      end = int(m.group(1)) * 60 + int(m.group(2))
      begin *= 1_000_000_000 / TIME_GRANULARITY
      end *= 1_000_000_000 / TIME_GRANULARITY
      if MERGE_ATRIO_LABELS and "AT_" in label:
        label = "A"
      times.append((begin,end,label))
WALK_TIMES_DATA = pd.DataFrame({'begin': [t[0] for t in times],
                  'end': [t[1] for t in times],
                  'label': [t[2] for t in times]})
WALK_TIMES_DATA

,begin,end,label
0,0.0,20.0,SS
1,21.0,40.0,TMA
2,41.0,60.0,DC
3,61.0,77.0,DG
4,77.0,110.0,TS
5,111.0,125.0,AH
6,126.0,169.0,SA
7,170.0,220.0,DF
8,221.0,240.0,CN
9,241.0,252.0,SN


## Map labels 

In [27]:
time_2_label = []
for r in WALK_TIMES_DATA.itertuples():
    t_begin = int(r.begin)
    t_end = int(r.end)
    for _ in range(t_end - t_begin+1):
        time_2_label.append(r.label)
len(time_2_label)

540

In [28]:
vc = VideoCapture.VideoInput(DPARAM[DEVICE]['video'])
next_time = 0
time_limit = len(time_2_label) - 1
print('Time:', vc.frame_count / vc.fps)
dataset_tsv_file = open('vision_walk_dataset_'+DEVICE+'.tsv','w')
print('label', 'set', 'time', 'source', 'image', 'video', sep='\t', file=dataset_tsv_file)
while next_time < time_limit:
    _, frame = vc.next_frame()
    label = time_2_label[next_time]
    directory = 'video_data/walk/%s/%s' % (DEVICE,label)
    os.makedirs(directory, exist_ok=True)
    image_file = directory + "/%04d_%05d.jpeg" % (next_time, vc.current_frame)
    cv2.imwrite(image_file, frame)
    print(label, 'test', next_time,  DEVICE, image_file, vc.video_file, sep='\t', file=dataset_tsv_file)
    print(next_time, label, image_file, vc.current_frame, sep='\t')
    next_time += 1
    while vc.current_time < next_time:
        _ = vc.next_frame()
dataset_tsv_file.close()

Time: 549.2425111111111
0	SS	video_data/walk/Xiaomi/SS/0000_00000.jpeg	0
1	SS	video_data/walk/Xiaomi/SS/0001_00030.jpeg	30
2	SS	video_data/walk/Xiaomi/SS/0002_00060.jpeg	60
3	SS	video_data/walk/Xiaomi/SS/0003_00090.jpeg	90
4	SS	video_data/walk/Xiaomi/SS/0004_00120.jpeg	120
5	SS	video_data/walk/Xiaomi/SS/0005_00150.jpeg	150
6	SS	video_data/walk/Xiaomi/SS/0006_00180.jpeg	180
7	SS	video_data/walk/Xiaomi/SS/0007_00210.jpeg	210
8	SS	video_data/walk/Xiaomi/SS/0008_00240.jpeg	240
9	SS	video_data/walk/Xiaomi/SS/0009_00270.jpeg	270
10	SS	video_data/walk/Xiaomi/SS/0010_00300.jpeg	300
11	SS	video_data/walk/Xiaomi/SS/0011_00330.jpeg	330
12	SS	video_data/walk/Xiaomi/SS/0012_00360.jpeg	360
13	SS	video_data/walk/Xiaomi/SS/0013_00390.jpeg	390
14	SS	video_data/walk/Xiaomi/SS/0014_00420.jpeg	420
15	SS	video_data/walk/Xiaomi/SS/0015_00450.jpeg	450
16	SS	video_data/walk/Xiaomi/SS/0016_00480.jpeg	480
17	SS	video_data/walk/Xiaomi/SS/0017_00509.jpeg	509
18	SS	video_data/walk/Xiaomi/SS/0018_00539.jpeg	539
19	

157	SA	video_data/walk/Xiaomi/SA/0157_04701.jpeg	4701
158	SA	video_data/walk/Xiaomi/SA/0158_04731.jpeg	4731
159	SA	video_data/walk/Xiaomi/SA/0159_04761.jpeg	4761
160	SA	video_data/walk/Xiaomi/SA/0160_04791.jpeg	4791
161	SA	video_data/walk/Xiaomi/SA/0161_04821.jpeg	4821
162	SA	video_data/walk/Xiaomi/SA/0162_04851.jpeg	4851
163	SA	video_data/walk/Xiaomi/SA/0163_04881.jpeg	4881
164	SA	video_data/walk/Xiaomi/SA/0164_04911.jpeg	4911
165	SA	video_data/walk/Xiaomi/SA/0165_04941.jpeg	4941
166	SA	video_data/walk/Xiaomi/SA/0166_04970.jpeg	4970
167	SA	video_data/walk/Xiaomi/SA/0167_05000.jpeg	5000
168	SA	video_data/walk/Xiaomi/SA/0168_05030.jpeg	5030
169	SA	video_data/walk/Xiaomi/SA/0169_05060.jpeg	5060
170	SA	video_data/walk/Xiaomi/SA/0170_05090.jpeg	5090
171	DF	video_data/walk/Xiaomi/DF/0171_05120.jpeg	5120
172	DF	video_data/walk/Xiaomi/DF/0172_05150.jpeg	5150
173	DF	video_data/walk/Xiaomi/DF/0173_05180.jpeg	5180
174	DF	video_data/walk/Xiaomi/DF/0174_05210.jpeg	5210
175	DF	video_data/walk/Xiaom

310	HA	video_data/walk/Xiaomi/HA/0310_09282.jpeg	9282
311	HA	video_data/walk/Xiaomi/HA/0311_09312.jpeg	9312
312	HA	video_data/walk/Xiaomi/HA/0312_09342.jpeg	9342
313	HA	video_data/walk/Xiaomi/HA/0313_09372.jpeg	9372
314	HA	video_data/walk/Xiaomi/HA/0314_09401.jpeg	9401
315	HA	video_data/walk/Xiaomi/HA/0315_09431.jpeg	9431
316	HA	video_data/walk/Xiaomi/HA/0316_09461.jpeg	9461
317	HA	video_data/walk/Xiaomi/HA/0317_09491.jpeg	9491
318	HA	video_data/walk/Xiaomi/HA/0318_09521.jpeg	9521
319	HA	video_data/walk/Xiaomi/HA/0319_09551.jpeg	9551
320	HA	video_data/walk/Xiaomi/HA/0320_09581.jpeg	9581
321	HA	video_data/walk/Xiaomi/HA/0321_09611.jpeg	9611
322	HA	video_data/walk/Xiaomi/HA/0322_09641.jpeg	9641
323	HA	video_data/walk/Xiaomi/HA/0323_09671.jpeg	9671
324	HA	video_data/walk/Xiaomi/HA/0324_09701.jpeg	9701
325	HA	video_data/walk/Xiaomi/HA/0325_09731.jpeg	9731
326	HA	video_data/walk/Xiaomi/HA/0326_09761.jpeg	9761
327	HA	video_data/walk/Xiaomi/HA/0327_09791.jpeg	9791
328	HA	video_data/walk/Xiaom

452	AT_I2	video_data/walk/Xiaomi/AT_I2/0452_13533.jpeg	13533
453	AT_I2	video_data/walk/Xiaomi/AT_I2/0453_13563.jpeg	13563
454	AT_I2	video_data/walk/Xiaomi/AT_I2/0454_13593.jpeg	13593
455	AT_I2	video_data/walk/Xiaomi/AT_I2/0455_13623.jpeg	13623
456	AT_I2	video_data/walk/Xiaomi/AT_I2/0456_13653.jpeg	13653
457	AT_I2	video_data/walk/Xiaomi/AT_I2/0457_13683.jpeg	13683
458	AT_I2	video_data/walk/Xiaomi/AT_I2/0458_13713.jpeg	13713
459	AT_I2	video_data/walk/Xiaomi/AT_I2/0459_13743.jpeg	13743
460	AT_I2	video_data/walk/Xiaomi/AT_I2/0460_13773.jpeg	13773
461	AT_I2	video_data/walk/Xiaomi/AT_I2/0461_13803.jpeg	13803
462	AT_I2	video_data/walk/Xiaomi/AT_I2/0462_13833.jpeg	13833
463	AT_I2	video_data/walk/Xiaomi/AT_I2/0463_13862.jpeg	13862
464	AT_I2	video_data/walk/Xiaomi/AT_I2/0464_13892.jpeg	13892
465	AT_I2	video_data/walk/Xiaomi/AT_I2/0465_13922.jpeg	13922
466	AT_I2	video_data/walk/Xiaomi/AT_I2/0466_13952.jpeg	13952
467	AT_I2	video_data/walk/Xiaomi/AT_I2/0467_13982.jpeg	13982
468	AT_I2	video_data/wal